$ \theta = \{\psi, \sigma_1, \sigma_2\} $

$p_{\theta}(s_{t+1}|s_t, a_t) = \{_{pdf(x_{t+1}, \; (x_t+a_t^0), \; \theta_2) \; \times \; pdf(y_{t+1}, \; (y_t+a_t^1), \; \theta_2) \; if \; x_t >= \theta_0}^{pdf(x_{t+1}, \; (x_t+a_t^0), \; \theta_1) \; \times \; pdf(y_{t+1}, \; (y_t+a_t^1), \; \theta_1) \; if \; x_t < \theta_0}$

In [12]:
import numpy as np
import random

theta = [5, 2, 4]

def transiction(s, a, p=2):
    return np.array(s) + np.array(a) + np.random.normal(0, p, 2)

def pickModel(s, theta):
    psi, *m = theta
    x,_ = s
    return m[int(x >= psi)]

def generate(s, n, theta):   
    for _ in range(n):
        a = random.choice([(1,0), (-1,0), (0,1), (0,-1)])
        yield s, a
        s = transiction(s, a, pickModel(s, theta))

s_0 = (1,1)
O = [(s, a) for (s, a) in generate(s_0, 100, theta)]
O

[((1, 1), (1, 0)),
 (array([3.95218613, 2.51070932]), (0, -1)),
 (array([-0.71340399, -2.95685681]), (0, 1)),
 (array([-4.417886  , -3.29214511]), (1, 0)),
 (array([-5.61129041, -1.86457798]), (1, 0)),
 (array([-2.35759381, -2.6355726 ]), (0, -1)),
 (array([-1.6054336 , -2.56177237]), (0, 1)),
 (array([-0.26629409,  1.24629043]), (0, -1)),
 (array([-2.38207098,  1.12319154]), (-1, 0)),
 (array([-4.44492119,  0.95513588]), (0, 1)),
 (array([-5.96026554,  1.6555269 ]), (-1, 0)),
 (array([-9.90436861,  1.69353749]), (0, -1)),
 (array([-11.67902551,   1.2540445 ]), (1, 0)),
 (array([-7.89248626,  1.20249277]), (0, -1)),
 (array([-6.50464646,  2.29716939]), (1, 0)),
 (array([-2.77681646,  4.54712814]), (1, 0)),
 (array([-2.45561269,  6.8108771 ]), (1, 0)),
 (array([-4.54204735,  3.91584188]), (0, 1)),
 (array([-3.49036892,  4.36620011]), (0, -1)),
 (array([-6.12663044,  3.27603272]), (1, 0)),
 (array([-7.50190176,  4.18168552]), (0, -1)),
 (array([-7.79802635,  2.53194281]), (-1, 0)),
 (arr

$p_{\theta}(s_{t+1}|s_t, a_t) = pdf(s^x_{t+1}, (s^x_t + a_t^x), \theta_2) \; \times \; pdf(s^y_{t+1}, (s^y_t+a_t^y), \theta_2) \; \times \; \mathbb{I}_{s^x_t >= \theta_0} + pdf(s^x_{t+1}, (s^x_t+a_t^x), \theta_1) \; \times \; pdf(s^y_{t+1}, (s^y_t+a_t^y), \theta_1) \; \times \; \mathbb{I}_{s^x_t < \theta_0}$



$p_{\theta}(s_{t+1}|s_t, a_t) = pdf(s^x_{t+1}, (s^x_t+a_t^x), \theta_2) \; \times \; pdf(s^y_{t+1}, (s^y_t+a_t^y), \theta_2) \; \times \; \frac{1}{1+e^{\beta_{s_t}}}$ <br>
$ \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; $
$ + \; pdf(s^x_{t+1}, (s^x_t+a_t^x), \theta_1) \; \times \; pdf(s^y_{t+1}, (s^y_t+a_t^y), \theta_1) \; \times \; (1 - \frac{1}{1+e^{\beta_{s_t}}})$ 


$ \beta_{s_t} = f(s^x_t, s^y_t; \theta)$

$\cal{L}[\xi | \theta] = \sum_{t=0}^{\mathrm{T}-1} log \; p_{\theta}(s_{t+1}|s_t, a_t) - \gamma$

$ \gamma = \sum_t \sum_{s\;vizinho\;s_t} (1-p(s)) * p(s_t) + p(s) * (1-p(s_t)) $

$ p(s) =  $

$ \gamma = \sum_t \sum_{s\;vizinho\;s_t} KL(p(s), p(s_t))$ <br>
KL = Kullback-Leibler 

$ \gamma = \sum_t \sum_{s} KL(p(s), p(s_t)) * e^{-|s_t - s|} $

In [13]:
from scipy.stats import norm

def ll(O, theta, c=10):
    psi, sig1, sig2 = theta

    beta = lambda x,y, theta: c * (theta-x)
    p = lambda x_,y_, x,y, ax,ay: ( 
        norm.pdf(x_, (x + ax), sig2)  
        * norm.pdf(y_, (y + ay), sig2) 
        * (1/(1 + np.e **beta(x,y, psi)))
        +
        norm.pdf(x_, (x + ax), sig1) 
        * norm.pdf(y_, (y + ay), sig1)
        * (1 - (1/(1 + np.e **beta(x,y, psi))))
    )

    ps = lambda x, y: x+y #?
    d = lambda p1, p2: (1-p1)*p2 + p1*(1-p2)

    S = set([(x,y) for ((x,y), _) in O])
    gamma = np.sum([d(ps(x,y), ps(xt,yt)) for t,((xt,yt),_) in enumerate(O) for (x,y) in S if (abs(x-xt)<1.5 and abs(y-yt)<1.5)])
    return np.sum([np.log(p(*st, *O[t-1][0], *O[t-1][1])) for t,(st,_) in enumerate(O) if 0 < t]) - gamma

ll(O, (5, 2, 4), c=10)

446124.7741454962

In [14]:
# best_theta = 0
# best_ll = -np.inf

# for psi in range(10):
#     for p1 in range(10):
#         for p2 in range(10):
#             theta = (psi+1, p1+1, p2+1)
#             new_ll = ll(O, theta)
#             if new_ll > best_ll:
#                 best_ll = new_ll
#                 best_theta = theta

# print(best_theta, best_ll)

C:\Users\guilherme.albarrans\AppData\Local\Temp\ipykernel_57128\2691689637.py:10: RuntimeWarning: overflow encountered in scalar power
  * (1/(1 + np.e **beta(x,y, psi)))
C:\Users\guilherme.albarrans\AppData\Local\Temp\ipykernel_57128\2691689637.py:14: RuntimeWarning: overflow encountered in scalar power
  * (1 - (1/(1 + np.e **beta(x,y, psi))))


(2, 2, 5) 446127.3225310683


In [30]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.DoubleTensor)
a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.DoubleTensor)

p = Variable(torch.Tensor([5, 2, 4]), requires_grad=True)
beta = 10

# loglike = torch.sum(normal.Normal(s[:,0] + a[:,0], torch.where(s[:,0] >= p[0], p[2], p[1])).log_prob(s_[:,0]) + normal.Normal(s[:,1] + a[:,1], torch.where(s[:,0] >= p[0], p[2], p[1])).log_prob(s_[:,1]))
loglike = torch.sum(
        torch.log(
        torch.exp(normal.Normal(s[:,0] + a[:,0], p[2]).log_prob(s_[:,0]))
        * torch.exp(normal.Normal(s[:,1] + a[:,1], p[2]).log_prob(s_[:,1]))
        * torch.sigmoid(-beta * (p[0]-s[:,0]))
        + 
        torch.exp(normal.Normal(s[:,0] + a[:,0], p[1]).log_prob(s_[:,0]))
        * torch.exp(normal.Normal(s[:,1] + a[:,1], p[1]).log_prob(s_[:,1]))
        * (1 - torch.sigmoid(-beta * (p[0]-s[:,0])))
    )
)

loglike.backward()
gradiente = p.grad.clone()
print('like: ', loglike)
# gradiente.numpy()

like:  tensor(-440.3043, dtype=torch.float64, grad_fn=<SumBackward0>)


In [5]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.DoubleTensor)
a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.DoubleTensor)

p = Variable(torch.randn(3), requires_grad=True)
beta = 10

while p[1] <= 0 or p[2] <= 0:
    p = Variable(torch.randn(3), requires_grad=True)

e_margin = 1e-6
learning_rate = 0.002
e = - 1e+10
for t in range(10000):
    # print(p)
    
    loglike = torch.sum(
        torch.log(
            torch.exp(normal.Normal(s[:,0] + a[:,0], p[2]).log_prob(s_[:,0]))
            * torch.exp(normal.Normal(s[:,1] + a[:,1], p[2]).log_prob(s_[:,1]))
            * torch.sigmoid(-beta * (p[0]-s[:,0]))
            + 
            torch.exp(normal.Normal(s[:,0] + a[:,0], p[1]).log_prob(s_[:,0]))
            * torch.exp(normal.Normal(s[:,1] + a[:,1], p[1]).log_prob(s_[:,1]))
            * (1 - torch.sigmoid(-beta * (p[0]-s[:,0])))
        )
    )
        
    loglike.backward()
    
    new_p = p.data + learning_rate * p.grad.data
    # print(t, loglike.data.numpy(),new_p, p, p.grad.data)
    if (np.abs(e) - np.abs(loglike.data.numpy())) <= e_margin:
    # if torch.all(torch.lt((new_p - p.data), torch.Tensor([0.0001,0.0001,0.0001]))):
        break
    else:
        if not new_p.isnan().any():
            p.data = new_p
            p.grad.data.zero_()
            e = loglike.data.numpy()
        else:
            break


print("loglike  =", loglike.data.numpy(), "theta =", p.data.numpy())

loglike  = -439.8474988484604 p = [5.2120705 1.9280896 4.0618114]


In [219]:
def estimate(beta = 1.5, init_p=None):
    s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.DoubleTensor)
    a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
    s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.DoubleTensor)

    if init_p is not None:
        p = p = torch.nn.parameter.Parameter(torch.tensor(init_p))
    else:
        p = torch.nn.parameter.Parameter(torch.Tensor(3).uniform_(1, 10))
    print('init_p: ', p)

    e_margin = 1e-6
    learning_rate = 0.002
    e = - 1e+10
    for t in range(10000):
        # print(p)
    
        loglike = torch.sum(
            torch.log(
                torch.exp(normal.Normal(s[:,0] + a[:,0], p[2]).log_prob(s_[:,0]))
                * torch.exp(normal.Normal(s[:,1] + a[:,1], p[2]).log_prob(s_[:,1]))
                * torch.sigmoid(-beta * (p[0]-s[:,0]))
                + 
                torch.exp(normal.Normal(s[:,0] + a[:,0], p[1]).log_prob(s_[:,0]))
                * torch.exp(normal.Normal(s[:,1] + a[:,1], p[1]).log_prob(s_[:,1]))
                * (1 - torch.sigmoid(-beta * (p[0]-s[:,0])))
            )
        )
            
        loglike.backward()
        
        new_p = p.data + learning_rate * p.grad.data
        # print(t, loglike.data.numpy(),new_p, p, p.grad.data)
        if (np.abs(e) - np.abs(loglike.data.numpy())) <= e_margin:
        # if torch.all(torch.lt((new_p - p.data), torch.Tensor([0.0001,0.0001,0.0001]))):
            break
        else:
            if not new_p.isnan().any():
                p.data = new_p
                p.grad.data.zero_()
                e = loglike.data.numpy()
            else:
                break

    # print("loglike  =", loglike.data.numpy(), "theta =", p.data.numpy())

    return p


estimate(2)

init_p:  Parameter containing:
tensor([7.4658, 7.9085, 7.8982], requires_grad=True)


Parameter containing:
tensor([5.5656, 1.9258, 4.2426], requires_grad=True)

In [225]:
def estimate(beta = 1.5, init_p=None):
    s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.DoubleTensor)
    a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
    s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.DoubleTensor)

    if init_p is not None:
        p = torch.nn.parameter.Parameter(torch.tensor(init_p))
    else:
        p = torch.nn.parameter.Parameter(torch.Tensor(3).uniform_(1, 10))
        # p = torch.nn.parameter.Parameter(torch.randn(3))
        # with torch.no_grad():
        #         p[:] = p.clamp(min=1)
        # while p[0] <= 0 or p[1] <= 0 or p[2] <= 0:
        #     p = Variable(torch.randn(3), requires_grad=True)
    # print('init_p: ', p)

    loglike = lambda: -torch.sum(
            torch.log(
            torch.exp(normal.Normal(s[:,0] + a[:,0], p[2]).log_prob(s_[:,0]))
            * torch.exp(normal.Normal(s[:,1] + a[:,1], p[2]).log_prob(s_[:,1]))
            * torch.sigmoid(-beta * (p[0]-s[:,0]))
            + 
            torch.exp(normal.Normal(s[:,0] + a[:,0], p[1]).log_prob(s_[:,0]))
            * torch.exp(normal.Normal(s[:,1] + a[:,1], p[1]).log_prob(s_[:,1]))
            * (1 - torch.sigmoid(-beta * (p[0]-s[:,0])))
        )
    )

    # optim = torch.optim.SGD([p], lr=1e-2, momentum=0.9)
    optim = torch.optim.SGD([p], lr=1e-2)

    for i in range(1000):
        # print(i, p)
        ll = loglike()
        optim.zero_grad()
        ll.backward()
        optim.step() 

        with torch.no_grad():
            p[:] = p.clamp(min=1e-1)

    return p


estimate(2)

Parameter containing:
tensor([5.5514, 1.9257, 4.2273], requires_grad=True)

In [233]:
start = [1., 1., 1.]

for i in range(10):
    print(i*.5, estimate(i*.5, start))

0.0 Parameter containing:
tensor([1.0000, 2.4884, 2.4884], requires_grad=True)
0.5 Parameter containing:
tensor([8.2470, 1.9428, 4.9115], requires_grad=True)
1.0 Parameter containing:
tensor([6.5869, 1.9313, 4.5599], requires_grad=True)
1.5 Parameter containing:
tensor([5.8309, 1.9270, 4.3175], requires_grad=True)
2.0 Parameter containing:
tensor([5.5514, 1.9257, 4.2273], requires_grad=True)
2.5 Parameter containing:
tensor([5.4274, 1.9251, 4.1834], requires_grad=True)
3.0 Parameter containing:
tensor([5.3500, 1.9250, 4.1500], requires_grad=True)
3.5 Parameter containing:
tensor([5.2903, 1.9254, 4.1201], requires_grad=True)
4.0 Parameter containing:
tensor([5.2436, 1.9259, 4.0949], requires_grad=True)
4.5 Parameter containing:
tensor([5.2107, 1.9265, 4.0764], requires_grad=True)


In [253]:
for i in range(10):
    n = [[round(n, 2) for n in estimate(i*.5).data.tolist()] for _ in range(5)]
    print(i*.5, *n)

0.0 [2.37, 3.19, 1.62] [4.57, 1.62, 3.19] [5.14, 1.62, 3.19] [6.27, 1.62, 3.19] [5.37, 1.62, 3.19]
0.5 [8.25, 1.94, 4.91] [8.28, 1.94, 4.92] [8.22, 1.94, 4.9] [8.33, 1.94, 4.94] [8.27, 1.94, 4.92]
1.0 [6.59, 1.93, 4.56] [6.59, 1.93, 4.56] [6.59, 1.93, 4.56] [6.59, 1.93, 4.56] [6.59, 1.93, 4.56]
1.5 [5.83, 1.93, 4.32] [5.83, 1.93, 4.32] [5.83, 1.93, 4.32] [5.83, 1.93, 4.32] [5.83, 1.93, 4.32]
2.0 [5.55, 1.93, 4.23] [5.55, 1.93, 4.23] [5.55, 1.93, 4.23] [5.55, 1.93, 4.23] [5.55, 1.93, 4.23]
2.5 [5.43, 1.93, 4.18] [5.43, 1.93, 4.18] [5.43, 1.93, 4.18] [5.43, 1.93, 4.18] [5.43, 1.93, 4.18]
3.0 [5.35, 1.93, 4.15] [5.35, 1.93, 4.15] [5.35, 1.93, 4.15] [5.35, 1.93, 4.15] [5.35, 1.93, 4.15]
3.5 [8.47, 2.19, 4.66] [5.29, 1.93, 4.12] [5.29, 1.93, 4.12] [5.29, 1.93, 4.12] [8.47, 2.19, 4.66]
4.0 [5.24, 1.93, 4.09] [5.24, 1.93, 4.09] [5.24, 1.93, 4.09] [5.24, 1.93, 4.09] [5.24, 1.93, 4.09]
4.5 [8.52, 2.19, 4.64] [8.52, 2.19, 4.64] [8.52, 2.19, 4.64] [5.21, 1.93, 4.08] [5.21, 1.93, 4.08]
